In [ ]:
# Install nibabel library, if not done yet (for reading gii files)
!pip install nibabel
# Install openpyxl, if not done yet (for saving Excel file)
!pip install openpyxl

In [1]:
import numpy as np
import pandas as pd
import nibabel as nb
import os

In [2]:
#Data location
#noninjured.L.ab
baseFolder  = 'C:\\RaghusWork\\Data\\GarciaPNAS2018_example\\noninjured.L.ab'
youngerFile = 'YAS.LLR.surf.gii'
olderRegFile = 'configincaltrelaxforward.anat.LLR.reg.surf.gii'

In [3]:
#Read data
img = nb.load(os.path.join(baseFolder, youngerFile))
younger_coords = img.agg_data('NIFTI_INTENT_POINTSET')
younger_triangles = img.agg_data('NIFTI_INTENT_TRIANGLE')

In [ ]:
img = nb.load(os.path.join(baseFolder, olderRegFile))
olderReg_coords = img.agg_data('NIFTI_INTENT_POINTSET')
olderReg_triangles = img.agg_data('NIFTI_INTENT_TRIANGLE')
print(np.array_equal(younger_triangles, olderReg_triangles))
print(np.array_equal(younger_coords,olderReg_coords))

In [ ]:
#compute growth vectors
growthVector = olderReg_coords - younger_coords

In [ ]:
#Converting to pandas dataframe
younger_coords = pd.DataFrame(younger_coords)
olderReg_coords = pd.DataFrame(olderReg_coords)
younger_triangles = pd.DataFrame( younger_triangles)
growthVector = pd.DataFrame( growthVector)

In [ ]:
# Co-efficient computation
a0 = []; a1 = []; a2 = [];
b0 = []; b1 = []; b2 = [];
c0 = []; c1 = []; c2 = [];
for index, edges in younger_triangles.iterrows():
    #if (index>2):
    #    break
    vertex0 = younger_coords.iloc[edges[0]].to_numpy()
    vertex1 = younger_coords.iloc[edges[1]].to_numpy()
    vertex2 = younger_coords.iloc[edges[2]].to_numpy()
    normal = np.cross(vertex1-vertex0, vertex2-vertex0)
    G0 = growthVector.iloc[edges[0]].to_numpy()
    G1 = growthVector.iloc[edges[1]].to_numpy()
    G2 = growthVector.iloc[edges[2]].to_numpy()
    n2 = np.dot(normal,normal)
    a = np.cross((G1[0]-G0[0])*normal,(vertex0-vertex2)) + np.cross((G2[0]-G0[0])*normal,(vertex1-vertex0))
    a = a/n2
    a0.append(a[0]); a1.append(a[1]); a2.append(a[2]);
    b = np.cross((G1[1]-G0[1])*normal,(vertex0-vertex2)) + np.cross((G2[1]-G0[1])*normal,(vertex1-vertex0))
    b = b/n2
    b0.append(b[0]); b1.append(b[1]); b2.append(b[2]);
    c = np.cross((G1[2]-G0[2])*normal,(vertex0-vertex2)) + np.cross((G2[2]-G0[2])*normal,(vertex1-vertex0))
    c = c/n2
    c0.append(c[0]); c1.append(c[1]); c2.append(c[2]);
    #print(edges[0],edges[1],edges[2])  
    #print(vertex0,vertex1,vertex2,normal,n2)
    #print(a,b,c)

In [ ]:
#Saving results to Excel file
with pd.ExcelWriter('noninjured_L_ab.xlsx', engine='openpyxl') as writer:
    younger_coords.to_excel(writer, sheet_name='Week27Toward31-vertices')
    olderReg_coords.to_excel(writer, sheet_name='Week31For27-vertices')
    younger_triangles.to_excel(writer, sheet_name='Week27Toward31-triangles')
    outDf = pd.DataFrame({'a0': a0, 'a1': a1, 'a2': a2, 'b0': b0, 'b1': b1, 'b2': b2, 'c0': c0, 'c1': c1, 'c2': c2})
    outDf.to_excel(writer, sheet_name='co-efficients') 

In [ ]:
#Display of growth vectors
import matplotlib.pyplot as plt
fig = plt.figure()
ax = plt.axes(projection="3d")
stride = 50
x = younger_coords[::stride,0]
y = younger_coords[::stride,1]
z = younger_coords[::stride,2]
u = growthVector[::stride,0]
v = growthVector[::stride,1]
w = growthVector[::stride,2]
ax.quiver(x, y, z, u, v, w, length=2., arrow_length_ratio = 2, zorder=3, color='black', normalize= True) #, width=0.007, headwidth=3., headlength=4.) 
plt.show()

In [14]:
import numpy as np
from collections import defaultdict

def check_orientation(vertices, triangles):
    def calculate_normal(v0, v1, v2):
        edge1 = v1 - v0
        edge2 = v2 - v0
        return np.cross(edge1, edge2)

    # Create a mapping from each vertex to the list of triangles that share that vertex
    vertex_to_triangles = defaultdict(list)
    for i, triangle_indices in enumerate(triangles):
        for vertex_index in triangle_indices:
            vertex_to_triangles[vertex_index].append(i)

    is_correctly_oriented = True

    for i, triangle_indices in enumerate(triangles):
        v0, v1, v2 = vertices[triangle_indices]
        triangle_normal = calculate_normal(v0, v1, v2)

        # Check orientation of the triangle with its neighbors
        neighbor_indices = set()
        for vertex_index in triangle_indices:
            neighbor_indices.update(vertex_to_triangles[vertex_index])
        neighbor_indices.remove(i)  # Remove current triangle from neighbors
        for neighbor_index in neighbor_indices:
            neighbor_v0, neighbor_v1, neighbor_v2 = vertices[triangles[neighbor_index]]
            neighbor_normal = calculate_normal(neighbor_v0, neighbor_v1, neighbor_v2)
            if np.dot(triangle_normal, neighbor_normal) < 0:
                is_correctly_oriented = False
                break

    return is_correctly_oriented

# Example usage
vertices = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]])
triangles = np.array([[0, 1, 2], [1, 3, 2]])

result = check_orientation(vertices, triangles) #True
print(result)


True


In [15]:
# Example usage
vertices = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0]])
triangles = np.array([[0, 1, 2], [1, 3, 2], [0, 2, 3]])

result = check_orientation(vertices, triangles)
print(result)

False


In [9]:
import numpy as np

def is_orientable(vertices, triangles):
    # Create a dictionary to store edge orientations
    edge_orientations = {}

    # Iterate over each triangle in the mesh
    for triangle in triangles:
        for i in range(3):
            # Get the indices of the current edge vertices
            v0_idx = triangle[i]
            v1_idx = triangle[(i + 1) % 3]

            # Compute the edge direction
            edge_direction = vertices[v1_idx] - vertices[v0_idx]

            # Normalize the edge direction
            edge_length = np.linalg.norm(edge_direction)
            edge_direction = edge_direction/float(edge_length) if edge_length != 0 else 1

            # Check if the edge is already in the dictionary
            if (v0_idx, v1_idx) in edge_orientations:
                # Check if the orientations are opposite
                if np.dot(edge_orientations[(v0_idx, v1_idx)], edge_direction) > 0:
                    return False
            else:
                # Add the edge to the dictionary
                edge_orientations[(v0_idx, v1_idx)] = edge_direction

    return True

# Example usage
vertices = [
    np.array([0, 0, 0]),
    np.array([1, 0, 0]),
    np.array([0, 1, 0]),
    np.array([0, 0, 1])
]

triangles = [
    [0, 1, 2],  # Triangle 1
    [0, 2, 3],  # Triangle 2
    [0, 3, 1]   # Triangle 3
]

print(is_orientable(vertices, triangles))  # Output: True

True


In [11]:
import numpy as np

def check_orientability(vertices, triangles):
    """
    Check if all triangles on a surface mesh are correctly oriented by testing if
    common edges between triangles have opposite directions.

    Parameters:
    - vertices (numpy array): Array of vertices in the mesh.
    - triangles (numpy array): Array of triangles in the mesh, where each row
      represents a triangle by referencing the indices of its three vertices.

    Returns:
    - orientable (bool): True if all triangles are correctly oriented, False otherwise.
    """

    def edge_direction(v1, v2):
        """
        Calculate the direction of an edge defined by two vertices.
        """
        return (v2 - v1) / np.linalg.norm(v2 - v1)

    orientable = True

    # Create a dictionary to store the indices of triangles sharing an edge
    edge_map = {}
    for i, triangle in enumerate(triangles):
        for j in range(3):
            edge = tuple(sorted([triangle[j], triangle[(j + 1) % 3]]))
            if edge not in edge_map:
                edge_map[edge] = []
            edge_map[edge].append(i)

    # Check if common edges between triangles have opposite directions
    for edge, triangles in edge_map.items():
        if len(triangles) == 2:
            triangle1, triangle2 = triangles
            v1, v2 = vertices[edge[0]], vertices[edge[1]]
            edge_direction1 = edge_direction(v1, v2)

            if not triangle1 or not triangle2:
                continue

            v1 = vertices[triangle1[(np.where(triangle1 == edge[0])[0][0] + 1) % 3]]
            v2 = vertices[triangle1[np.where(triangle1 == edge[0])[0][0]]]
            edge_direction2 = edge_direction(v1, v2)

            if np.dot(edge_direction1, edge_direction2) > 0:
                orientable = False
                break

            v1 = vertices[triangle2[np.where(triangle2 == edge[0])[0][0]]]
            v2 = vertices[triangle2[(np.where(triangle2 == edge[0])[0][0] + 1) % 3]]
            edge_direction2 = edge_direction(v1, v2)

            if np.dot(edge_direction1, edge_direction2) > 0:
                orientable = False
                break

    return orientable

In [12]:
result = check_orientability(younger_coords, younger_triangles)
#result = is_orientable(younger_coords, younger_triangles)
print(result)

IndexError: index 0 is out of bounds for axis 0 with size 0